<a href="https://colab.research.google.com/github/tankTopTaro/Deep-Learning-with-Python/blob/main/00_tensorflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook will cover some fundamental concepts of tensors using TensorFlow

Thing to covers:
* Introduction to tensors
* Getting informations from tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up regular Python functions)
* Using GPUs with TensorFlow (or TPUs)
* Exercises to try for yourself!


# Introduction to Tensors

In [1]:
# Import TensorFlow
import tensorflow as tf


print(tf.__version__)

2.8.2


In [2]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [3]:
# Check the number of dimensions of a tensor
# (ndim stands for number of dimensions)
scalar.ndim

0

In [4]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [5]:
# Check the dimension of vector
vector.ndim

1

In [6]:
# Create a matrix
matrix = tf.constant([[10, 7],
                      [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [7]:
# Check the dimension of matrix
matrix.ndim

2

In [8]:
# Create another matrix
# Specify the data type with dtype parameter
another_matrix = tf.constant([[10., 7.],
                             [3., 2.],
                             [8., 9.]], dtype=tf.float16)
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [9]:
# Check the dimension of another_matrix
another_matrix.ndim

2

In [10]:
# Create a tensor
tensor = tf.constant([[[1, 2, 3],
                       [4, 5, 6]],
                      [[7, 8, 9],
                       [10, 11, 12]],
                      [[13, 14, 15],
                       [16, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [11]:
# Check the dimension of the tensor
tensor.ndim

3

* Scalar: a single number
* Vector: a number with direction
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimensional array of numbers (a 0-D tensor is a scalar, a 1-D tensor is a vector

### Creating tensors with `tf.Variable`

In [12]:
tf.Variable

tensorflow.python.ops.variables.Variable

In [16]:
# Create the same tensors with tf.Variable() as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [19]:
# Try to change an element in changeable_tensor and unchangeable_tensor
changeable_tensor[0] = 7
changeable_tensor


TypeError: ignored

In [20]:
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [21]:
unchangeable_tensor[0] = 7
unchangeable_tensor

TypeError: ignored

In [22]:
unchangeable_tensor[0].assign(7)
unchangeable_tensor

AttributeError: ignored

🔑 **Note:** Rarely in practice will you need to decide whether to use `tf.constant` or `tf.Variable` to create tensors, as TensorFlow does this for you. However, if in doubt, use `tf.constant` and change it later if needed.

### Creating random tensors
Random tensors are tensors of some arbitrary size which contain random numbers.

In [25]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(7)    # set seed for reproducibility
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(7)
random_2 = random_2.normal(shape=(3, 2))

# Are they equal
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.28785667],
        [-0.8757901 , -0.08857018],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.28785667],
        [-0.8757901 , -0.08857018],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### Shuffle the order of elements in a tensor
Shuffling is valuable for when you want to shuffle data so the inherent order doesn't affect learning

In [34]:
# Create a non-shuffled tensor
not_shuffled = tf.constant([[10, 7],
                            [3, 4],
                            [2, 5]])

# Shuffle the non-shuffled tensor
tf.random.set_seed(42)
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

⚒ **Exercise:** Read through TensorFlow documentation on random seed generation: https://www.tensorflow.org/api_docs/python/tf/random/set_seed and practice writing 5 random tensors

In [39]:
# if global seed nor the operation seed is set, we get different results for 
## every call to the random op and every re-run of the program
print(tf.random.uniform([1]))  # generates 'A1'
print(tf.random.uniform([1]))  # generates 'A2'

tf.Tensor([0.6765541], shape=(1,), dtype=float32)
tf.Tensor([0.2962978], shape=(1,), dtype=float32)


In [43]:
tf.random.set_seed(1234)  # Global Seed
print(tf.random.uniform([1]))  # generates 'A1'
print(tf.random.uniform([1]))  # generates 'A2'

tf.Tensor([0.5380393], shape=(1,), dtype=float32)
tf.Tensor([0.3253647], shape=(1,), dtype=float32)
